In [0]:
silver_layer_df = spark.table('az_adb_simbus_training.imdb_project_schema.silver_popular_actor_directors')

#display(silver_layer_df)

In [0]:
from pyspark.sql.functions import col, avg, sum as sum_,round,count
from pyspark.sql import functions as F

gold_aggregate_actors_movies_df = silver_layer_df.filter('role == "actor"').groupBy("person_name")\
    .agg(
        count("movie_title").alias("movie_count"),
        round(avg("average_rating"), 2).alias("average_rating"),
        sum_("num_votes").alias("total_votes"),
        F.first("role").alias("role")
    )

gold_aggregate_directors_movies_df = silver_layer_df.filter('role == "director"').groupBy("person_name")\
    .agg(
        count("movie_title").alias("movie_count"),
        round(avg("average_rating"), 2).alias("average_rating"),
        sum_("num_votes").alias("total_votes"),
        F.first("role").alias("role")
    )

#filtering out the person who has a count more than 10 movies
filtered_gold_actors_movies_df = gold_aggregate_actors_movies_df.filter(col("movie_count") > 10)

filtered_gold_directors_movies_df = gold_aggregate_directors_movies_df.filter(col("movie_count") > 10)

#display(filtered_gold_actors_movies_df)
#display(filtered_gold_directors_movies_df)




In [0]:
filtered_gold_actors_movies_df.write.format("delta").mode("overwrite").saveAsTable("az_adb_simbus_training.imdb_project_schema.golden_popular_actors")

filtered_gold_directors_movies_df.write.format("delta").mode("overwrite").saveAsTable("az_adb_simbus_training.imdb_project_schema.golden_popular_directors")


In [0]:
# from pyspark.sql.functions import col, avg, sum as sum_,round,count
gold_aggregate_year_movies_df = (silver_layer_df.groupBy("person_name","start_year","role")
    .agg(
        count("movie_title").alias("movies_in_year"),
    ).select('role','start_year','person_name','movies_in_year'))

# display(gold_aggregate_year_movies_df.limit(100))    

In [0]:
gold_aggregate_year_movies_df.write.format("delta").mode("overwrite").saveAsTable("az_adb_simbus_training.imdb_project_schema.gold_movies_per_year")